In [1]:
from dash import Dash, dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import viridis

In [2]:
df = pd.read_csv('Combined.csv')
duration_mins_secs = [str(f"{i}:{j}") for i, j in zip([int(i//1) for i in list(df['Duration'])], [int(round((i%1) * 60, 2)) for i in list(df['Duration'])])]
df['Track Duration'] = duration_mins_secs
df.sample(5)

,Unnamed: 0,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,...,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Playlist,Track Duration
51,51,Ghost,spotify:track:6I3mqTwhRpn34SLVafSH7G,https://open.spotify.com/track/6I3mqTwhRpn34SL...,['Justin Bieber'],Justice,88,2.55,153.960,2,...,0.741,-5.569,0.0478,0.18500,0.000029,0.415,0.441,4,Playlist 2,2:33
76,76,Wildest Dreams,spotify:track:3fVnlF4pGqWI9flVENcT28,https://open.spotify.com/track/3fVnlF4pGqWI9fl...,['Taylor Swift'],1989,81,3.67,140.060,8,...,0.664,-7.417,0.0741,0.07090,0.005600,0.106,0.467,4,Playlist 2,3:40
63,63,Love Again,spotify:track:4rPkN1FMzQyFNP9cLUGIIB,https://open.spotify.com/track/4rPkN1FMzQyFNP9...,['Dua Lipa'],Future Nostalgia,76,4.30,115.982,11,...,0.667,-4.668,0.0339,0.00173,0.000029,0.100,0.468,4,Playlist 2,4:18
25,25,The Man Who Can't Be Moved,spotify:track:3nqqDo8CcCLke3ZoTgiOKf,https://open.spotify.com/track/3nqqDo8CcCLke3Z...,['The Script'],The Script,77,4.02,99.894,10,...,0.629,-5.051,0.0279,0.43700,0.000000,0.097,0.308,4,Playlist 1,4:1
68,68,The Nights,spotify:track:0ct6r3EGTcMLPtrXHDvVjc,https://open.spotify.com/track/0ct6r3EGTcMLPtr...,['Avicii'],The Days / Nights,87,2.94,125.983,6,...,0.835,-5.298,0.0433,0.01660,0.000000,0.249,0.654,4,Playlist 2,2:56


In [3]:
p1 = pd.read_csv('Playlist1.csv')
p1 = pd.DataFrame(p1[['Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Speechiness', 'Valence']].mean()).T
p2 = pd.read_csv('Playlist2.csv')
p2 = pd.DataFrame(p2[['Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Speechiness', 'Valence']].mean()).T
p = pd.concat([p1, p2], ignore_index=True)
p

,Acousticness,Danceability,Energy,Instrumentalness,Speechiness,Valence
0,0.323171,0.49882,0.54488,0.051619,0.035736,0.346190
1,0.134577,0.64820,0.71412,0.012604,0.061402,0.557222


In [4]:
artists = pd.read_csv('All Artists.csv')
artists['Index'] = artists.index
artists.sample(5)

,Name,ID,Genres,Followers,Popularity,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2,Index
70,Maren Morris,6WY7D3jk8zTrHtmkqqo5GI,"['classic texas country', 'contemporary country']",777453,68,Playlist 2,1,0,1,70
22,OneRepublic,5Pwc4xIPtQLFEnJriah9YJ,"['piano rock', 'pop']",16556557,80,Both,4,1,3,22
34,Gary Lightbody,3fCQnw6VSudXBTN5d2QP2z,['british singer-songwriter'],9339,52,Playlist 1,1,1,0,34
49,Clean Bandit,6MDME20pz9RveH9rEXvrOM,"['pop', 'uk dance', 'uk funky']",5361605,71,Playlist 2,1,0,1,49
79,Anne-Marie,1zNqDE7qDGCsyzJwohVaoX,['pop'],11467826,76,Playlist 2,1,0,1,79


In [5]:
genres = pd.read_csv('Genres.csv')
genres['Index'] = genres.index
genres.sample(5)

,Genre,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2,Index
78,art pop,Playlist 2,1,0,1,78
20,pop,Both,47,7,40,20
73,blues rock,Playlist 1,1,1,0,73
83,uk post-punk,Both,2,1,1,83
6,modern rock,Both,13,9,4,6


In [6]:
collabs = pd.read_csv('All Collaborations.csv')
collabs.sample(5)

,Artist 1,Artist 2,Popular Collab,URL,Popularity,Artist From
114,Martin Garrix,Dua Lipa,Scared to Be Lonely,https://open.spotify.com/track/3ebXMykcMXOcLeJ...,80.0,Artists from Same Playlist
17,Maroon 5,Tainy,Button (feat. Anuel AA & Tainy),https://open.spotify.com/track/6FDKYM0lBf6zqrg...,50.0,Artists in Different Playlists
3,Taylor Swift,Shawn Mendes,Lover (Remix) [feat. Shawn Mendes],https://open.spotify.com/track/3i9UVldZOE0aD0J...,77.0,Artists in Different Playlists
61,Clean Bandit,Zara Larsson,Symphony (feat. Zara Larsson),https://open.spotify.com/track/72gv4zhNvRVdQA0...,80.0,Artists from Same Playlist
11,The Killers,Clean Bandit,Caution - Clean Bandit Remix,https://open.spotify.com/track/73A4FC5eIihO0sw...,34.0,Artists in Different Playlists


In [7]:
artists_songs = pd.read_csv("Common Artist Songs.csv")
artists_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Artist
2,Anti-Hero,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://open.spotify.com/track/0V3wPSX9ygBnCm8...,['Taylor Swift'],Midnights,92,3.34,97.008,4,1,0.637,0.643,-6.571,0.0519,0.1300,0.000002,0.1420,0.533,4,Taylor Swift
4,"""Slut!"" (Taylor's Version) (From The Vault)",spotify:track:71BqAINEnezjQfxE4VuJfq,https://open.spotify.com/track/71BqAINEnezjQfx...,['Taylor Swift'],1989 (Taylor's Version),91,3.01,77.983,7,1,0.629,0.413,-13.178,0.0550,0.3450,0.000076,0.0603,0.306,4,Taylor Swift
10,Perfect,spotify:track:0tgVpDi06FyKpA1z0VMD4v,https://open.spotify.com/track/0tgVpDi06FyKpA1...,['Ed Sheeran'],÷ (Deluxe),89,4.39,95.050,8,1,0.599,0.448,-6.312,0.0232,0.1630,0.000000,0.1060,0.168,3,Ed Sheeran
15,RUNAWAY,spotify:track:07fbDnkKdZGk1gLvknxrns,https://open.spotify.com/track/07fbDnkKdZGk1gL...,['OneRepublic'],RUNAWAY,81,2.39,163.033,3,0,0.624,0.684,-5.507,0.0513,0.0336,0.000000,0.0771,0.750,4,OneRepublic
16,Sunshine,spotify:track:5r43qanLhUCdBj8HN3fa6B,https://open.spotify.com/track/5r43qanLhUCdBj8...,['OneRepublic'],Sunshine,80,2.73,140.069,5,1,0.697,0.875,-4.621,0.0340,0.1370,0.000000,0.2800,0.774,4,OneRepublic


In [8]:
genre_songs = pd.read_csv("Common Genre Songs.csv")
genre_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Genre
222,Ghosts Again,spotify:track:3p5JD79PQP737dyEY2hnmf,https://open.spotify.com/track/3p5JD79PQP737dy...,['Depeche Mode'],Memento Mori,65,3.98,115.010,6,0,0.589,0.748,-4.767,0.0259,0.000438,0.034900,0.1600,0.3340,4,new wave
249,Blackbird - Remastered 2009,spotify:track:5jgFfDIR6FR0gvlA56Nakr,https://open.spotify.com/track/5jgFfDIR6FR0gvl...,['The Beatles'],The Beatles (Remastered),78,2.31,93.699,7,1,0.686,0.127,-14.361,0.0342,0.754000,0.000014,0.0573,0.3720,4,rock
135,Skyfall,spotify:track:6VObnIkLVruX4UVyxWhlqm,https://open.spotify.com/track/6VObnIkLVruX4UV...,['Adele'],Skyfall,86,4.77,75.881,0,0,0.346,0.552,-6.864,0.0282,0.417000,0.000000,0.1140,0.0789,4,uk pop
347,Always Been You,spotify:track:53UMRr4KOgZP5Ins0aGlhs,https://open.spotify.com/track/53UMRr4KOgZP5In...,['Jessie Murph'],drowning,75,2.20,137.120,3,0,0.655,0.572,-7.666,0.0535,0.646000,0.000002,0.0774,0.4030,4,singer-songwriter pop
309,The Blonde,spotify:track:72cGBEqu7RitIOoACXYjfR,https://open.spotify.com/track/72cGBEqu7RitIOo...,['TV Girl'],French Exit,79,3.80,87.692,1,0,0.501,0.547,-7.961,0.0243,0.225000,0.912000,0.2880,0.2250,4,pov: indie


In [9]:
new_songs = pd.read_csv("New Playlist.csv")
duration_mins_secs = [str(f"{i}:{j}") for i, j in zip([int(i//1) for i in list(new_songs['Duration'])], [int(round((i%1) * 60, 2)) for i in list(new_songs['Duration'])])]
new_songs['Track Duration'] = duration_mins_secs
new_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Track Duration
272,Take Me to Church,spotify:track:1CS7Sd1u5tWkstBhpssyjP,https://open.spotify.com/track/1CS7Sd1u5tWkstB...,['Hozier'],Hozier (Expanded Edition),80,4.03,128.945,4,0,0.566,0.664,-5.303,0.0464,0.6340,0.000000,0.1160,0.437,4,4:1
163,Everybody Wants To Rule The World,spotify:track:4RvWPyQ5RL0ao9LPZeSouE,https://open.spotify.com/track/4RvWPyQ5RL0ao9L...,['Tears For Fears'],Songs From The Big Chair (Super Deluxe Edition),89,4.19,112.067,7,1,0.645,0.795,-12.095,0.0527,0.3470,0.003890,0.1040,0.535,4,4:11
238,One Kiss (with Dua Lipa),spotify:track:7ef4DlsgrMEH11cDZd32M6,https://open.spotify.com/track/7ef4DlsgrMEH11c...,"['Calvin Harris', 'Dua Lipa']",One Kiss (with Dua Lipa),86,3.58,123.994,9,0,0.791,0.862,-3.240,0.1100,0.0370,0.000022,0.0814,0.592,4,3:34
81,This Love,spotify:track:6ECp64rv50XVz93WvxXMGF,https://open.spotify.com/track/6ECp64rv50XVz93...,['Maroon 5'],Songs About Jane: 10th Anniversary Edition,84,3.44,95.051,5,0,0.712,0.862,-4.612,0.0378,0.0525,0.000000,0.0930,0.809,4,3:26
324,Easy On Me,spotify:track:46IZ0fSY2mpAiktS3KOqds,https://open.spotify.com/track/46IZ0fSY2mpAikt...,['Adele'],30,83,3.74,141.981,5,1,0.604,0.366,-7.519,0.0282,0.5780,0.000000,0.1330,0.130,4,3:44


In [10]:
cols = ['Popularity', 'Tempo', 'Danceability', 'Energy', 'Acousticness', 'Valence']
scatter_cols = ['Acousticness', 'Danceability', 'Energy', 'Tempo', 'Valence']
hist_cols = ['Popularity', 'Tempo', 'Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveliness', 'Valence']

---

In [11]:
app1 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app1.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='left', children=[
            dbc.Label("Comparing Playlists - CDF Plots"),
            dcc.Graph(id="histogram")
        ]),
        dbc.Col(width=2, align='right', children=[
            html.Div([
                html.Label('Field'),
                dcc.Dropdown(id='field', value=hist_cols[0], options=[{'label': col, 'value': col} for col in hist_cols])
            ]),
            html.Div([
                html.Label('Bins'),
                dcc.Slider(id='nbins', min=5, max=200, marks=None, step=1, value=10)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, marks=None, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app1.callback(
    Output("histogram", "figure"), 
    [Input("field", "value"),
     Input('nbins', 'value'), 
     Input('opacity', 'value')]
)

def update_chart(field, nbins, opacity):
    fig = px.histogram(df, x=field, histfunc='count', nbins=nbins, color='Playlist', color_discrete_sequence=['limegreen', 'fuchsia'], opacity=opacity, cumulative=True, width=1400, height=600)
    fig.update_layout(
        barmode='overlay',
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app1.run(port=8855, debug=True)

---

In [12]:
app2 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app2.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Scatterplot"),
            dcc.Graph(id="scatter")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('X-Axis'),
                dcc.Dropdown(id='xax', value=scatter_cols[0], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Y-Axis'),
                dcc.Dropdown(id='yax', value=scatter_cols[1], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Size'),
                dcc.Dropdown(id='size', value=scatter_cols[2], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app2.callback(
    Output("scatter", "figure"), 
    [Input("xax", "value"),
     Input('yax', 'value'),
     Input('size', 'value'),
     Input('opacity', 'value')]
)

def update_figure(xax, yax, size, opacity):
    hover_dict = {xax: False, yax:False, size:False, 'Playlist':False, 'Album':True, 'Artist(s)':True, 'Track Duration':True, 'Popularity':True}
    fig = px.scatter(df, x=xax, y=yax, size=size, color='Playlist', trendline='ols', color_discrete_sequence=['limegreen', 'fuchsia'], hover_name="Track_Name", hover_data=hover_dict, size_max=25, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity=opacity)
    return fig

app2.run(port=8856, debug=True)

---

In [13]:
app3 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app3.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Radar Plot"),
            dcc.Graph(id="radar"),
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5, marks=None, step=None)
            ]),
        ])
    ])
], fluid=True)

@app3.callback(
    Output("radar", "figure"),
    Input('opacity', 'value')
)

def update_figure(opacity):
    trace1 = go.Scatterpolar(
        r = list(p.iloc[0, :]),
        theta = list(p.columns),
        fill = 'toself',
        fillcolor = 'limegreen',
        line = dict(color='black', width=1),
        name='Playlist 1'
    )
    trace2 = go.Scatterpolar(
        r = list(p.iloc[1, :]),
        theta = list(p.columns),
        fill = 'toself',
        fillcolor = 'fuchsia',
        line = dict(color='black', width=1),
        name = 'Playlist 2'
    )
    layout = go.Layout(
        polar = dict(
            radialaxis = dict(
                visible = True,
                range = [0, 1]
            )
        ),
        showlegend = True,
        height = 600,
        width = 1400,
        plot_bgcolor = 'rgba(0, 0, 0, 0)'
    )
    fig = go.Figure(data=[trace1, trace2], layout=layout)
    fig.update_layout(
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity = opacity)
    return fig

app3.run(port=8857, debug=True)

---

In [14]:
app4 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app4.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Top Artists"),
            dcc.Graph(id="bubble")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value='Followers', options=[{'label': col, 'value': col} for col in ['Followers', 'Popularity']])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app4.callback(
    Output("bubble", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value'),
     Input('opacity', 'value')]
)

def update_figure(measure, topn, opacity):
    hover_dict = {measure: False, 'Index':False, 'Name':False, 'Playlist':False, 'Genres':True, 'Popularity':True, 'Followers':True, 'Total Count':False}
    fig = px.scatter(artists.sort_values(by=measure, ascending=False).head(topn), x='Index', y=measure, text='Name', size='Total Count', size_max=100, color='Playlist', color_discrete_map={'Playlist 1':'limegreen', 'Playlist 2':'fuchsia', 'Both':'yellow'}, hover_name="Name", hover_data=hover_dict, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app4.run(port=8858, debug=True)

---

In [15]:
app5 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app5.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Top Genres"),
            dcc.Graph(id="bubble_too")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app5.callback(
    Output("bubble_too", "figure"),
     [Input('topn', 'value'),
     Input('opacity', 'value')]
)

def update_figure(topn, opacity):
    hover_dict = {'Index':False, 'Genre':False, 'Playlist':False, 'Total Count':True}
    fig = px.scatter(genres.sort_values(by='Total Count', ascending=False).head(topn), x='Total Count', y='Index', text='Genre', size='Total Count', size_max=100, color='Playlist', color_discrete_map={'Playlist 1':'limegreen', 'Playlist 2':'fuchsia', 'Both':'yellow'}, hover_name="Genre", hover_data=hover_dict, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel=dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app5.run(port=8859, debug=True)

---

In [16]:
app6 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app6.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Collaboration Network"),
            dcc.Graph(id="network")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ])
        ])
    ])
], fluid=True)

@app6.callback(
    Output("network", "figure"),
     Input('opacity', 'value')
)

def update_figure(opacity):

    G = nx.Graph()
    for i, j, k, l, m in zip(list(artists['Name']), list(artists['Genres']), list(artists['Popularity']), list(artists['Followers']), list(artists['Playlist'])):
        G.add_node(i)
        G.nodes[i]['Genres'] = j
        G.nodes[i]['Popularity'] = k
        G.nodes[i]['Followers'] = l
        G.nodes[i]['Playlist'] = m
    for i, j, k, l, m in zip(list(collabs['Artist 1']), list(collabs['Artist 2']), list(collabs['Popular Collab']), list(collabs['Popularity']), list(collabs['Artist From'])):
        G.add_edge(i, j)
        G.edges[(i, j)]['Popular Collab'] = k
        G.edges[(i, j)]['Popularity'] = l
        G.edges[(i, j)]['Artist From'] = m
    #nx.write_gexf(G, 'Collaboration Graph.gexf')

    pos = nx.spring_layout(G, k=0.99)
    pos = nx.rescale_layout_dict(pos, 1000)
    for n in G.nodes:
        G.nodes[n]['pos'] = pos[n]

    edge_x = []
    edge_y = []
    xtext = []
    ytext = []
    edgetext = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        xtext.append((x0 + x1)/2)
        ytext.append((y0 + y1)/2)
        edgetext.append(f"Artist 1: {edge[0]}, \nArtist 2: {edge[1]},\n Collaboration: {G.edges[edge]['Popular Collab']}")
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, 
        y=edge_y,
        line = dict(
            width = 1, 
            color = '#888'
        ),
        hoverinfo='text',
        mode='lines'
    )
    edge_trace.text = edgetext

    eweights_trace = go.Scatter(
        x = xtext,
        y = ytext, 
        mode = 'markers',
        marker = dict(
            size = 3,
            color = '#888',
        ),
        text = edgetext,
        hoverinfo = 'text'
    )

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)
    
    node_trace = go.Scatter(
        x = node_x, 
        y = node_y,
        mode = 'markers',
        hoverinfo = 'text',
        marker = dict(
            color = [],
            size = [],
            line_width = 0.5
        )
    )

    pd = {'Playlist 1':'limegreen', 'Playlist 2':'fuchsia', 'Both':'yellow'}

    node_size = []
    node_color = []
    node_text = []
    for n in G.nodes:
        node_size.append((nx.degree_centrality(G)[n] * 500) + 10)
        node_color.append(pd[G.nodes[n]['Playlist']])
        node_text.append(f"Name: {n},\nGenres: {G.nodes[n]['Genres']}")

    node_trace.marker.size = node_size
    node_trace.marker.color = node_color
    node_trace.text = node_text

    layout = go.Layout(
        showlegend = False,
        hovermode = 'closest',
        xaxis = dict(
            showgrid = False, 
            zeroline = False, 
            showticklabels = False
        ),
        yaxis = dict(
            showgrid = False, 
            zeroline = False, 
            showticklabels = False
        ),
        height=1100,
        width=1400,
        plot_bgcolor='rgba(0, 0, 0, 0)'
    )

    fig = go.Figure(data=[edge_trace, eweights_trace, node_trace], layout=layout)
    fig.update_layout(
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity = opacity)
    return fig

app6.run(port=8860, debug=True)

---

In [17]:
app7 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app7.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Wall of Recommendations - Common Genres"),
            dcc.Graph(id="treemap")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
        ])
    ])
], fluid=True)

@app7.callback(
    Output("treemap", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value')]
)

def update_figure(measure, topn):
    hover_dict = {'Artist(s)':True, measure:False, 'Track_Name':False}
    fig = px.treemap(
        genre_songs.sort_values(by=measure, ascending=False).head(topn), 
        path = ['Common Genre', 'Track_Name'],
        values = measure,
        color = measure,
        color_continuous_scale=px.colors.sequential.Purp,
        hover_name = 'Track_Name',
        hover_data = hover_dict,
    )
    fig.update_layout(
        width=1400,
        height=600,
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        ),
    )
    return fig

app7.run(port=8861, debug=True)

---

In [18]:
app8 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app8.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Wall of Recommendations - Common Artists"),
            dcc.Graph(id="treemap_too")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
        ])
    ])
], fluid=True)

@app8.callback(
    Output("treemap_too", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value')]
)

def update_figure(measure, topn):
    hover_dict = {'Artist(s)':True, measure:False, 'Track_Name':False}
    fig = px.treemap(
        artists_songs.sort_values(by=measure, ascending=False).head(topn), 
        path = ['Common Artist', 'Track_Name'],
        values = measure,
        color = measure,
        color_continuous_scale=px.colors.sequential.Purp,
        hover_name = 'Track_Name',
        hover_data = hover_dict
    )
    fig.update_layout(
        width=1400,
        height=600,
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app8.run(port=8862, debug=True)

---

In [21]:
app9 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app9.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Scatterplot"),
            dcc.Graph(id="scatter")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('X-Axis'),
                dcc.Dropdown(id='xax', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Y-Axis'),
                dcc.Dropdown(id='yax', value=cols[1], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Size'),
                dcc.Dropdown(id='size', value=cols[2], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Color'),
                dcc.Dropdown(id='color', value=cols[3], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5, step=None, marks=None)
            ]),
        ])
    ])
], fluid=True)

@app9.callback(
    Output("scatter", "figure"), 
    [Input("xax", "value"),
     Input('yax', 'value'),
     Input('size', 'value'),
     Input('color', 'value'),
     Input('opacity', 'value')]
)

def update_figure(xax, yax, size, color, opacity):
    hover_dict = {xax: False, yax:False, size:False, color:False, 'Album':True, 'Artist(s)':True, 'Track Duration':True, 'Popularity':True}
    fig = px.scatter(new_songs, x=xax, y=yax, size=size, color=color, trendline='ols', color_continuous_scale='Purp', hover_name="Track_Name", hover_data=hover_dict, size_max=15, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity=opacity)
    return fig

app9.run(port=8863, debug=True)

---

In [14]:
app10 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app10.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='left', children=[
            dbc.Label("New Playlist - CDF Plot"),
            dcc.Graph(id="histogram")
        ]),
        dbc.Col(width=2, align='right', children=[
            html.Div([
                html.Label('Field'),
                dcc.Dropdown(id='field', value=hist_cols[0], options=[{'label': col, 'value': col} for col in hist_cols])
            ]),
            html.Div([
                html.Label('Bins'),
                dcc.Slider(id='nbins', min=5, max=200, marks=None, step=1, value=10)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, marks=None, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app10.callback(
    Output("histogram", "figure"), 
    [Input("field", "value"),
     Input('nbins', 'value'), 
     Input('opacity', 'value')]
)

def update_chart(field, nbins, opacity):
    fig = px.histogram(new_songs, x=field, histfunc='count', nbins=nbins, opacity=opacity, cumulative=True, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app10.run(port=8864, debug=True)

---